In [1]:
import numpy as np
import scipy 
import scipy.stats
from scipy import stats
import seaborn as sns
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import itertools
from itertools import groupby
import pickle
import os
import math
from sympy import S, symbols
from collections import Counter

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('fivethirtyeight')
#sns.mpl.rcParams['figure.figsize'] = (16, 10)

# Directory to store pickled dataframes
directory = '/Users/dianaow/Documents/formula-1-race-data/dataframes/'

In [426]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import cross_validation
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, classification_report, recall_score, confusion_matrix
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.under_sampling import TomekLinks
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [5]:
palette = {"Ultra soft":"#9b59b6", 
           "Super soft":"#ff5745", 
           "Soft":"#edee3d", 
           "Medium":"#cccccc", 
           "Hard":"#ff9932",
           "Intermediate":"#00f164",
           "Wet":"#85c8fb",
           "None": "#fabebe"}

plt.style.use("dark_background")

In [3]:
def read_from_pickle(directory, filename):
    df = pd.DataFrame()
    filepath = directory + filename
    with open(filepath, 'rb') as file:
        df = pickle.load(file)
            
    return df

In [33]:
df_stint1 = read_from_pickle(directory, "df_stint1.pickle")
df_stint1

,year,driverRef,stint,tyre,value,stint 1,stint 2,stint 3,stint 4,raceId,...,Super soft_3,Ultra soft_3,Wet_3,None_4,Soft_4,Super soft_4,Ultra soft_4,Wet_4,combi2.0,combi3.0
0,2016,hamilton,1,Super soft,16.0,16.0,41.0,0.0,0.0,948.0,...,0,0,0,1,0,0,0,0,0,0
4,2016,hamilton,1,Soft,1.0,1.0,4.0,1.0,15.0,950.0,...,1,0,0,0,1,0,0,0,0,0
8,2016,hamilton,1,Super soft,13.0,13.0,15.0,13.0,16.0,949.0,...,1,0,0,0,1,0,0,0,0,0
12,2016,hamilton,1,Super soft,17.0,17.0,36.0,0.0,0.0,951.0,...,0,0,0,1,0,0,0,0,0,0
16,2016,hamilton,1,Soft,1.0,1.0,0.0,0.0,0.0,952.0,...,0,0,0,1,0,0,0,0,1,0
20,2016,hamilton,1,Wet,31.0,31.0,47.0,0.0,0.0,953.0,...,0,0,0,1,0,0,0,0,0,1
24,2016,hamilton,1,Ultra soft,24.0,24.0,46.0,0.0,0.0,954.0,...,0,0,0,1,0,0,0,0,0,1
28,2016,hamilton,1,Super soft,15.0,15.0,36.0,0.0,0.0,955.0,...,0,0,0,1,0,0,0,0,0,0
32,2016,hamilton,1,Ultra soft,21.0,21.0,33.0,17.0,0.0,956.0,...,0,0,0,1,0,0,0,0,0,1
36,2016,hamilton,1,Wet,7.0,7.0,10.0,35.0,0.0,957.0,...,0,0,0,1,0,0,0,0,1,0


In [313]:
X_k = read_from_pickle(directory, "X_k.pickle")
clusters = read_from_pickle(directory, "clusters.pickle")
df_tyres = read_from_pickle(directory, "df_tyres.pickle")
drivers_2016 = read_from_pickle(directory, "drivers_2016.pickle")
drivers_2016t = read_from_pickle(directory, "drivers_2016t.pickle")

In [116]:
clusters16 = read_from_pickle(directory, "clusters16.pickle")

In [6]:
xl = pd.ExcelFile("./formula-1-race-data/Selected_Sets.xlsx", encoding='utf-8')
xl.sheet_names
selected_sets = xl.parse("Sheet10")

In [8]:
xl = pd.ExcelFile("./formula-1-race-data/Track_Information.xlsx", encoding='utf-8')
xl.sheet_names
track_info = xl.parse("Copy of Sheet12")

In [146]:
xl = pd.ExcelFile("/Users/dianaow/Downloads/F1_Tyre_data.xlsx")
xl.sheet_names
pirelli = xl.parse("Sheet7")

### Pre-processing before performing Classification

In [147]:
# SS, S, M = 1
# S, M, H = 2
# SS, S, US = 3

for i,row in pirelli.iterrows(): 
    if (pirelli.loc[i,'Super Soft'] == 'Super Soft') & (pirelli.loc[i,'Soft'] == 'Soft') & (pirelli.loc[i,'Medium'] == "Medium"):  
        pirelli.loc[i,'mandatory combi'] = float(1)
    elif (pirelli.loc[i,'Soft'] == "Soft") & (pirelli.loc[i,'Medium'] == "Medium") & (pirelli.loc[i,'Hard'] == "Hard"):
        pirelli.loc[i,'mandatory combi'] = float(2)
    elif (pirelli.loc[i,'Super Soft'] == 'Super Soft') & (pirelli.loc[i,'Soft'] == 'Soft') & (pirelli.loc[i,'Ultra Soft'] == 'Ultra Soft'):
        pirelli.loc[i,'mandatory combi'] = float(3)
    elif (pirelli.loc[i,'Super Soft'] == 'Super Soft') & (pirelli.loc[i,'Soft'] == 'Soft') & (pirelli.loc[i,'Medium'] != "Medium"): 
        pirelli.loc[i,'mandatory combi'] = float(4)
    elif (pirelli.loc[i,'Super Soft'] != 'Super Soft') & (pirelli.loc[i,'Soft'] == 'Soft') & (pirelli.loc[i,'Medium'] != "Medium"): 
        pirelli.loc[i,'mandatory combi'] = float(5)
    elif (pirelli.loc[i,'Super Soft'] != 'Soft') & (pirelli.loc[i,'Medium'] == "Medium") & (pirelli.loc[i,'Hard'] == "Hard"): 
        pirelli.loc[i,'mandatory combi'] = float(6)
    else:
        pirelli.loc[i,'mandatory combi'] = float(7)

In [542]:
pirelli.sort_values(['name', 'year'])

,year,name,Super Soft,Soft,Medium,Hard,Ultra Soft,mandatory combi
39,2015,Abu Dhabi Grand Prix,Super Soft,Soft,NaN,NaN,NaN,4.0
20,2016,Abu Dhabi Grand Prix,Super Soft,Soft,NaN,NaN,Ultra Soft,3.0
59,2017,Abu Dhabi Grand Prix,Super Soft,Soft,NaN,NaN,Ultra Soft,3.0
21,2015,Australian Grand Prix,NaN,Soft,Medium,NaN,NaN,7.0
0,2016,Australian Grand Prix,Super Soft,Soft,Medium,NaN,NaN,1.0
40,2017,Australian Grand Prix,Super Soft,Soft,NaN,NaN,Ultra Soft,3.0
28,2015,Austrian Grand Prix,Super Soft,Soft,NaN,NaN,NaN,4.0
8,2016,Austrian Grand Prix,Super Soft,Soft,NaN,NaN,Ultra Soft,3.0
48,2017,Austrian Grand Prix,Super Soft,Soft,NaN,NaN,Ultra Soft,3.0
7,2016,Azerbaijan Grand Prix,Super Soft,Soft,Medium,NaN,NaN,1.0


#### Ingredients of the classification model:
- Count of each tyre type selected for the race (teams select the number of tyres for their driver based on Pirelli's allocated mandatory tyre set)
- Cluster label based on the track's 2016 pit strategy records
- Cluster label based on k-means clustering result of track information (tyre stress rating, average track temperature)
- Driver-specific multiplier for each tyre type


In [131]:
clusters_new = clusters[['name', 'clusters']]
clusters_new = clusters_new.rename(columns={"clusters": "track info clusters"})

clusters16_new = clusters16.reset_index()[['year', 'name','clusters']]
clusters16_new = clusters16_new.rename(columns={"clusters": "clusters(pit strategy)"})

selected_sets = selected_sets.rename(columns={"Year":"year", "Race":"name", "Driver": "driverRef"})

# Merge df containing cluster labels for each race
# We assume that races will be clustered similarly in 2016 and 2017. 
dfc = pd.merge(clusters16_new, clusters_new, on='name', how='left')
dfc = dfc.drop(['year'], axis=1)
df_final = pd.merge(selected_sets, dfc, on=['name'], how='left')

In [333]:
drivers_2016pt = pd.pivot_table(drivers_2016t, values='ratio', index=['driverRef'], columns=['tyre']).reset_index()
drivers_2016pt.rename(columns={"Hard": "H_m", "Intermediate": "I_m", "Medium":"M_m", "Soft": "S_m", "Super soft": "SS_m", "Ultra soft":"US_m", "Wet": "W_m"}, inplace=True)

In [341]:
# Merge df containing driver-specific tyre degradation multiplier
df_final_1 = pd.merge(df_final, drivers_2016pt.reset_index(drop=True), on=['driverRef'], how='left')
df_final_1

,year,name,driverRef,Medium,Soft,Super Soft,Ultra soft,Hard,clusters(pit strategy),track info clusters,H_m,I_m,M_m,S_m,SS_m,US_m,W_m
0,2017,Brazilian Grand Prix,vettel,1.0,3.0,9.0,0.0,0.0,0,2,1.107796,1.023426,1.111487,1.176995,0.811017,0.821851,1.100406
1,2017,Brazilian Grand Prix,bottas,1.0,4.0,8.0,0.0,0.0,0,2,1.359567,1.011526,1.252126,1.054087,0.942624,0.680609,0.585670
2,2017,Brazilian Grand Prix,raikkonen,1.0,3.0,9.0,0.0,0.0,0,2,0.691182,0.916323,1.009876,1.124674,0.901477,0.999456,0.432438
3,2017,Brazilian Grand Prix,hamilton,1.0,4.0,8.0,0.0,0.0,0,2,0.976510,0.833021,0.851359,0.882515,1.063797,1.730973,1.568657
4,2017,Brazilian Grand Prix,verstappen,1.0,3.0,9.0,0.0,0.0,0,2,0.888318,0.791370,1.020470,0.911364,0.946199,1.341781,0.705081
5,2017,Brazilian Grand Prix,ricciardo,1.0,4.0,8.0,0.0,0.0,0,2,1.025336,0.694184,1.045744,1.010958,1.172284,1.074864,0.720563
6,2017,Brazilian Grand Prix,mass,1.0,2.0,10.0,0.0,0.0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017,Brazilian Grand Prix,alonso,1.0,3.0,9.0,0.0,0.0,0,2,1.044397,1.106728,0.677418,1.018829,1.069176,0.945672,1.080843
8,2017,Brazilian Grand Prix,perez,1.0,4.0,8.0,0.0,0.0,0,2,0.995572,0.791370,1.179297,1.215491,1.082196,0.544085,1.539827
9,2017,Brazilian Grand Prix,hulkenberg,1.0,2.0,10.0,0.0,0.0,0,2,0.711792,1.166230,1.037634,0.961426,1.072898,0.884462,0.974497


In [47]:
df_tyres1 = df_tyres[['year', 'name', 'driverRef', 'first set']]
df_tyres2 = df_tyres[['year', 'name', 'driverRef', 'second set']]
df_tyres3 = df_tyres[['year', 'name', 'driverRef', 'third set']]
df_tyres4 = df_tyres[['year', 'name', 'driverRef', 'fourth set']]

In [250]:
XY1 = pd.merge(df_final_1, df_tyres1, on=['year', 'name', 'driverRef'], how='left')

In [342]:
XY = pd.merge(df_final_1, df_tyres[['year', 'name', 'driverRef', 'first set', 'second set', 'third set', 'fourth set']], on=['year', 'name', 'driverRef'], how='left')

In [155]:
# Null values because pirelli did not publish information of selected tyres for these races, althoug they did release the mandatory tyre set info for the race.
# I will instead inpute these values with the average values of races assigned the same mandatory tyre sets. 
# Each race has mandatory 3 tyre sets, so if there are more than 3 avg values, only 3 can be selected. Base it off the previos' year's tyre choice

In [348]:
XY = pd.merge(XY, pirelli[['year', 'name', 'mandatory combi']], on=['year', 'name'], how='left')
XY.groupby("mandatory combi").agg({'Medium':'mean', 'Soft':'mean', 'Super Soft':'mean', 'Ultra soft':'mean', 'Hard':'mean'})

,Super Soft,Medium,Soft,Ultra soft,Hard
mandatory combi,,,,,
1.0,7.059701,1.567164,3.661692,0.000000,0.000000
2.0,0.000000,4.023077,7.138462,0.507692,1.330769
3.0,2.932258,0.000000,1.977419,8.090323,0.000000


In [256]:
# Find out which are the races with null values for driver selected sets
XY.drop_duplicates(subset=['year','name'], keep='first', inplace=False).sort_values("name")

,year,name,driverRef,Medium,Soft,Super Soft,Ultra soft,Hard,clusters(pit strategy),track info clusters,ratio,first set,second set,third set,fourth set,mandatory combi
3441,2016,Abu Dhabi Grand Prix,hamilton,0.0,4.0,2.0,7.0,0.0,1,0,0.447947,Ultra soft,Soft,Soft,None,3.0
1459,2017,Abu Dhabi Grand Prix,hamilton,0.0,2.0,2.0,9.0,0.0,1,0,0.447947,Ultra soft,Super soft,None,None,3.0
3342,2016,Australian Grand Prix,hamilton,1.0,6.0,6.0,0.0,0.0,1,0,0.447947,Super soft,Medium,None,None,1.0
1376,2017,Australian Grand Prix,hamilton,NaN,NaN,NaN,NaN,NaN,1,0,0.447947,Ultra soft,Soft,None,None,3.0
2542,2016,Austrian Grand Prix,hamilton,0.0,2.0,3.0,8.0,0.0,1,2,0.447947,Ultra soft,Soft,Soft,None,3.0
795,2017,Austrian Grand Prix,hamilton,0.0,2.0,3.0,8.0,0.0,1,2,0.447947,Super soft,Ultra soft,None,None,3.0
2642,2016,Azerbaijan Grand Prix,hamilton,1.0,4.0,8.0,0.0,0.0,1,0,0.447947,Super soft,Soft,None,None,1.0
878,2017,Azerbaijan Grand Prix,hamilton,1.0,4.0,8.0,0.0,0.0,1,0,0.447947,Super soft,Soft,Super soft,None,1.0
3242,2016,Bahrain Grand Prix,hamilton,1.0,6.0,6.0,0.0,0.0,1,0,0.447947,Super soft,Medium,Super soft,Soft,1.0
1293,2017,Bahrain Grand Prix,hamilton,NaN,NaN,NaN,NaN,NaN,1,0,0.447947,Super soft,Soft,Soft,None,1.0


In [343]:
def fillna_tyres(XY1):   
    for i, row in XY1.iterrows():
        if (XY1.loc[i,"clusters(pit strategy)"] == 1) & (XY1.loc[i,"name"] == "Australian Grand Prix") & (XY1.loc[i,"year"] == 2017):
            XY1.loc[i,"Super Soft"] =3 
            XY1.loc[i,"Ultra soft"] =8
            XY1.loc[i,"Soft"] =2
        elif (XY1.loc[i,"clusters(pit strategy)"] == 1) & (XY1.loc[i,"name"] == "Bahrain Grand Prix")  & (XY1.loc[i,"year"] == 2017):
            XY1.loc[i,"Super Soft"] =7
            XY1.loc[i,"Medium"] =2
            XY1.loc[i,"Soft"] = 4
        elif (XY1.loc[i,"clusters(pit strategy)"] == 1) & (XY1.loc[i,"name"] == "Italian Grand Prix")  & (XY1.loc[i,"year"] == 2016):
            XY1.loc[i,"Super Soft"] =7
            XY1.loc[i,"Medium"] =2
            XY1.loc[i,"Soft"] = 4
        elif (XY1.loc[i,"clusters(pit strategy)"] == 1) & (XY1.loc[i,"name"] == "Mexican Grand Prix")  & (XY1.loc[i,"year"] == 2016):
            XY1.loc[i,"Super Soft"] =7
            XY1.loc[i,"Medium"] =2
            XY1.loc[i,"Soft"] = 4
        elif (XY1.loc[i,"clusters(pit strategy)"] == 1) & (XY1.loc[i,"name"] == "Russian Grand Prix")  & (XY1.loc[i,"year"] == 2017):
            XY1.loc[i,"Super Soft"] =3 
            XY1.loc[i,"Ultra soft"] =8
            XY1.loc[i,"Soft"] =2
        elif (XY1.loc[i,"clusters(pit strategy)"] == 1) & (XY1.loc[i,"name"] == "Spanish Grand Prix")  & (XY1.loc[i,"year"] == 2017):
            XY1.loc[i,"Medium"] = 5
            XY1.loc[i,"Hard"] =1
            XY1.loc[i,"Soft"] = 7
            
    # Null values in first set belong to drivers who DNS or did not finish the stint
    # Inpute first sets with "None" indicating they failed to finish the stint
    XY1['first set'] = XY1['first set'].fillna("None")
    XY1['second set'] = XY1['second set'].fillna("None")
    XY1['third set'] = XY1['third set'].fillna("None")
    XY1['fourth set'] = XY1['fourth set'].fillna("None")

    # Fill null values for the tyre columns with 0. Null values indicate the tyre was not part of the mandatory set
    tyrecols = ["Medium", "Soft", "Super Soft", "Ultra soft", "Hard"]
    for i in tyrecols:
        XY1[i] = XY1[i].fillna(0)  
        
    # New drivers in 2017 (stroll, gasly, hartley) do not have any tyre degration multiplier. 
    # Inpute their multipler as 1.
    XY1["H_m"] = XY1["H_m"].fillna(1)
    XY1["I_m"] = XY1["I_m"].fillna(1)
    XY1["M_m"] = XY1["M_m"].fillna(1)
    XY1["S_m"] = XY1["S_m"].fillna(1)
    XY1["SS_m"] = XY1["SS_m"].fillna(1)
    XY1["US_m"] = XY1["US_m"].fillna(1)
    XY1["W_m"] = XY1["W_m"].fillna(1)
    
    return XY1

In [344]:
XY = fillna_tyres(XY)

In [350]:
XY.isnull().sum()

year                      0
name                      0
driverRef                 0
Medium                    0
Soft                      0
Super Soft                0
Ultra soft                0
Hard                      0
clusters(pit strategy)    0
track info clusters       0
H_m                       0
I_m                       0
M_m                       0
S_m                       0
SS_m                      0
US_m                      0
W_m                       0
first set                 0
second set                0
third set                 0
fourth set                0
mandatory combi           0
dtype: int64

#### Divide model by races allocated SS, S, M tyre set, S, M, H tyre set, SS, S, US tyre set
#### Further divide model by each stint. Assume that the maximum possible number of stints in a race is 4. In total, there are 12 separate classification models

In [527]:
# SS, S, M = 1
# S, M, H = 2
# SS, S, US = 3

XY1 = XY.drop(['second set', 'third set', 'fourth set'], axis=1)
XY1_c1 = XY1[XY1['mandatory combi'] == 1].reset_index()
XY1_c1 = XY1_c1.drop(["I_m", "US_m", "W_m", "H_m"], axis=1)
XY1_c2 = XY1[XY1['mandatory combi'] == 2].reset_index()
XY1_c2 = XY1_c2.drop(["I_m", "US_m", "W_m", "SS_m"], axis=1)
XY1_c3 = XY1[XY1['mandatory combi'] == 3].reset_index()
XY1_c3 = XY1_c3.drop(["I_m", "M_m", "W_m", "H_m"], axis=1)

XY2 = XY.drop(['first set', 'third set', 'fourth set'], axis=1)
XY2_c1 = XY2[XY2['mandatory combi'] == 1].reset_index()
XY2_c1 = XY2_c1.drop(["I_m", "US_m", "W_m", "H_m"], axis=1)
XY2_c2 = XY2[XY2['mandatory combi'] == 2].reset_index()
XY2_c2 = XY2_c2.drop(["I_m", "US_m", "W_m", "SS_m"], axis=1)
XY2_c3 = XY2[XY2['mandatory combi'] == 3].reset_index()
XY2_c3 = XY2_c3.drop(["I_m", "M_m", "W_m", "H_m"], axis=1)

XY3 = XY.drop(['second set', 'first set', 'fourth set'], axis=1)
XY3_c1 = XY3[XY3['mandatory combi'] == 1].reset_index()
XY3_c1 = XY3_c1.drop(["I_m", "US_m", "W_m", "H_m"], axis=1)
XY3_c2 = XY3[XY3['mandatory combi'] == 2].reset_index()
XY3_c2 = XY3_c2.drop(["I_m", "US_m", "W_m", "SS_m"], axis=1)
XY3_c3 = XY3[XY3['mandatory combi'] == 3].reset_index()
XY3_c3 = XY3_c3.drop(["I_m", "M_m", "W_m", "H_m"], axis=1)

XY4 = XY.drop(['second set', 'first set', 'third set'], axis=1)
XY4_c1 = XY4[XY4['mandatory combi'] == 1].reset_index()
XY4_c1 = XY4_c1.drop(["I_m", "US_m", "W_m", "H_m"], axis=1)
XY4_c2 = XY4[XY4['mandatory combi'] == 2].reset_index()
XY4_c2 = XY4_c2.drop(["I_m", "US_m", "W_m", "SS_m"], axis=1)
XY4_c3 = XY4[XY4['mandatory combi'] == 3].reset_index()
XY4_c3 = XY4_c3.drop(["I_m", "M_m", "W_m", "H_m"], axis=1)

In [580]:
XY1

,year,name,driverRef,Medium,Soft,Super Soft,Ultra soft,Hard,clusters(pit strategy),track info clusters,H_m,I_m,M_m,S_m,SS_m,US_m,W_m,first set,mandatory combi
0,2017,Brazilian Grand Prix,vettel,1.0,3.0,9.0,0.0,0.0,0,2,1.107796,1.023426,1.111487,1.176995,0.811017,0.821851,1.100406,Super soft,1.0
1,2017,Brazilian Grand Prix,bottas,1.0,4.0,8.0,0.0,0.0,0,2,1.359567,1.011526,1.252126,1.054087,0.942624,0.680609,0.585670,Super soft,1.0
2,2017,Brazilian Grand Prix,raikkonen,1.0,3.0,9.0,0.0,0.0,0,2,0.691182,0.916323,1.009876,1.124674,0.901477,0.999456,0.432438,Super soft,1.0
3,2017,Brazilian Grand Prix,hamilton,1.0,4.0,8.0,0.0,0.0,0,2,0.976510,0.833021,0.851359,0.882515,1.063797,1.730973,1.568657,Soft,1.0
4,2017,Brazilian Grand Prix,verstappen,1.0,3.0,9.0,0.0,0.0,0,2,0.888318,0.791370,1.020470,0.911364,0.946199,1.341781,0.705081,Super soft,1.0
5,2017,Brazilian Grand Prix,ricciardo,1.0,4.0,8.0,0.0,0.0,0,2,1.025336,0.694184,1.045744,1.010958,1.172284,1.074864,0.720563,Soft,1.0
6,2017,Brazilian Grand Prix,mass,1.0,2.0,10.0,0.0,0.0,0,2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,None,1.0
7,2017,Brazilian Grand Prix,alonso,1.0,3.0,9.0,0.0,0.0,0,2,1.044397,1.106728,0.677418,1.018829,1.069176,0.945672,1.080843,Super soft,1.0
8,2017,Brazilian Grand Prix,perez,1.0,4.0,8.0,0.0,0.0,0,2,0.995572,0.791370,1.179297,1.215491,1.082196,0.544085,1.539827,Super soft,1.0
9,2017,Brazilian Grand Prix,hulkenberg,1.0,2.0,10.0,0.0,0.0,0,2,0.711792,1.166230,1.037634,0.961426,1.072898,0.884462,0.974497,Super soft,1.0


### Classification of tyres to use for races assigned the mandatory tyre set of Super soft, Soft, Medium

#### For Combination 1, what tyre type is each driver going to use for Stint 1?

In [369]:
XY1_c1_16 = XY1_c1[XY1_c1['year'] == 2016].reset_index(drop=True)
X1_c1 = XY1_c1_16.drop(['year', 'name', 'driverRef', 'first set', 'mandatory combi'], axis=1)
Y1_c1 = XY1_c1_16['first set']

XY1_c1_17 = XY1_c1[XY1_c1['year'] == 2017].reset_index(drop=True)
X1_c1_17 = XY1_c1_17.drop(['year', 'name', 'driverRef', 'first set', 'mandatory combi'], axis=1)
Y1_c1_17 = XY1_c1_17['first set']

In [372]:
SS = StandardScaler()
X1_c1 = SS.fit_transform(X1_c1)
X1_c1_17 = SS.fit_transform(X1_c1_17)

In [361]:
XY1_c1['first set'].value_counts()

Super soft    263
Soft          121
None           12
Medium          6
Name: first set, dtype: int64

In [465]:
# Train multinomial logistic regression model
def logreg(dftest, X_train, X_test, Y_train, Y_test):
    mul_lr = LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(X_train, Y_train)

    print "Multinomial Logistic regression Train Accuracy :: ", accuracy_score(Y_train, mul_lr.predict(X_train))
    print "Multinomial Logistic regression Test Accuracy :: ", accuracy_score(Y_test, mul_lr.predict(X_test))
    
    p = pd.DataFrame(mul_lr.predict(X_test)).reset_index(drop=True)
    results = pd.concat([dftest, p], axis=1)
    
    return results

In [373]:
XY1_c1_res = logreg(XY1_c1_17, X1_c1, X1_c1_17, Y1_c1, Y1_c1_17)
XY1_c1_res

Multinomial Logistic regression Train Accuracy ::  0.657024793388
Multinomial Logistic regression Test Accuracy ::  0.60625


,index,year,name,driverRef,Medium,Soft,Super Soft,Ultra soft,Hard,clusters(pit strategy),track info clusters,M_m,S_m,SS_m,first set,mandatory combi,0
0,0,2017,Brazilian Grand Prix,vettel,1.0,3.0,9.0,0.0,0.0,0,2,1.111487,1.176995,0.811017,Super soft,1.0,Super soft
1,1,2017,Brazilian Grand Prix,bottas,1.0,4.0,8.0,0.0,0.0,0,2,1.252126,1.054087,0.942624,Super soft,1.0,Soft
2,2,2017,Brazilian Grand Prix,raikkonen,1.0,3.0,9.0,0.0,0.0,0,2,1.009876,1.124674,0.901477,Super soft,1.0,Soft
3,3,2017,Brazilian Grand Prix,hamilton,1.0,4.0,8.0,0.0,0.0,0,2,0.851359,0.882515,1.063797,Soft,1.0,Soft
4,4,2017,Brazilian Grand Prix,verstappen,1.0,3.0,9.0,0.0,0.0,0,2,1.020470,0.911364,0.946199,Super soft,1.0,Super soft
5,5,2017,Brazilian Grand Prix,ricciardo,1.0,4.0,8.0,0.0,0.0,0,2,1.045744,1.010958,1.172284,Soft,1.0,Soft
6,6,2017,Brazilian Grand Prix,mass,1.0,2.0,10.0,0.0,0.0,0,2,1.000000,1.000000,1.000000,None,1.0,Super soft
7,7,2017,Brazilian Grand Prix,alonso,1.0,3.0,9.0,0.0,0.0,0,2,0.677418,1.018829,1.069176,Super soft,1.0,Soft
8,8,2017,Brazilian Grand Prix,perez,1.0,4.0,8.0,0.0,0.0,0,2,1.179297,1.215491,1.082196,Super soft,1.0,Soft
9,9,2017,Brazilian Grand Prix,hulkenberg,1.0,2.0,10.0,0.0,0.0,0,2,1.037634,0.961426,1.072898,Super soft,1.0,Super soft


#### For Combination 1, what tyre type is each driver going to use for Stint 2?

In [531]:
XY2_c1['second set'].value_counts()

Soft          246
Medium         61
Super soft     57
None           38
Name: second set, dtype: int64

In [530]:
XY2_c1_16 = XY2_c1[XY2_c1['year'] == 2016].reset_index(drop=True)
X2_c1 = XY2_c1_16.drop(['year', 'name', 'driverRef', 'second set', 'mandatory combi'], axis=1)
Y2_c1 = XY2_c1_16['second set']

XY2_c1_17 = XY2_c1[XY2_c1['year'] == 2017].reset_index(drop=True)
X2_c1_17 = XY2_c1_17.drop(['year', 'name', 'driverRef', 'second set', 'mandatory combi'], axis=1)
Y2_c1_17 = XY2_c1_17['second set']

X2_c1 = SS.fit_transform(X2_c1)
X2_c1_17 = SS.fit_transform(X2_c1_17)

In [379]:
XY2_c1_res = logreg(XY2_c1_17, X2_c1, X2_c1_17, Y2_c1, Y2_c1_17)
XY2_c1_res

Multinomial Logistic regression Train Accuracy ::  0.623966942149
Multinomial Logistic regression Test Accuracy ::  0.5625


,index,year,name,driverRef,Medium,Soft,Super Soft,Ultra soft,Hard,clusters(pit strategy),track info clusters,M_m,S_m,SS_m,second set,mandatory combi,0
0,0,2017,Brazilian Grand Prix,vettel,1.0,3.0,9.0,0.0,0.0,0,2,1.111487,1.176995,0.811017,Soft,1.0,Medium
1,1,2017,Brazilian Grand Prix,bottas,1.0,4.0,8.0,0.0,0.0,0,2,1.252126,1.054087,0.942624,Soft,1.0,Medium
2,2,2017,Brazilian Grand Prix,raikkonen,1.0,3.0,9.0,0.0,0.0,0,2,1.009876,1.124674,0.901477,Soft,1.0,Medium
3,3,2017,Brazilian Grand Prix,hamilton,1.0,4.0,8.0,0.0,0.0,0,2,0.851359,0.882515,1.063797,Super soft,1.0,Medium
4,4,2017,Brazilian Grand Prix,verstappen,1.0,3.0,9.0,0.0,0.0,0,2,1.020470,0.911364,0.946199,Soft,1.0,Medium
5,5,2017,Brazilian Grand Prix,ricciardo,1.0,4.0,8.0,0.0,0.0,0,2,1.045744,1.010958,1.172284,Soft,1.0,Medium
6,6,2017,Brazilian Grand Prix,mass,1.0,2.0,10.0,0.0,0.0,0,2,1.000000,1.000000,1.000000,None,1.0,Medium
7,7,2017,Brazilian Grand Prix,alonso,1.0,3.0,9.0,0.0,0.0,0,2,0.677418,1.018829,1.069176,Soft,1.0,Medium
8,8,2017,Brazilian Grand Prix,perez,1.0,4.0,8.0,0.0,0.0,0,2,1.179297,1.215491,1.082196,Soft,1.0,Medium
9,9,2017,Brazilian Grand Prix,hulkenberg,1.0,2.0,10.0,0.0,0.0,0,2,1.037634,0.961426,1.072898,Soft,1.0,Medium


#### For Combination 1, what tyre type is each driver going to use for Stint 3?

In [536]:
XY3_c1['third set'].value_counts()

None          186
Soft           85
Super soft     72
Medium         59
Name: third set, dtype: int64

In [466]:
XY3_c1_16 = XY3_c1[XY3_c1['year'] == 2016].reset_index(drop=True)
X3_c1 = XY3_c1_16.drop(['year', 'name', 'driverRef', 'third set', 'mandatory combi'], axis=1)
Y3_c1 = XY3_c1_16['third set']

XY3_c1_17 = XY3_c1[XY3_c1['year'] == 2017].reset_index(drop=True)
X3_c1_17 = XY3_c1_17.drop(['year', 'name', 'driverRef', 'third set', 'mandatory combi'], axis=1)
Y3_c1_17 = XY3_c1_17['third set']

X3_c1 = SS.fit_transform(X3_c1)
X3_c1_17 = SS.fit_transform(X3_c1_17)

XY3_c1_res = logreg(XY3_c1_17, X3_c1, X3_c1_17, Y3_c1, Y3_c1_17)
XY3_c1_res

Multinomial Logistic regression Train Accuracy ::  0.47520661157
Multinomial Logistic regression Test Accuracy ::  0.34375


,index,year,name,driverRef,Medium,Soft,Super Soft,Ultra soft,Hard,clusters(pit strategy),track info clusters,M_m,S_m,SS_m,third set,mandatory combi,0
0,0,2017,Brazilian Grand Prix,vettel,1.0,3.0,9.0,0.0,0.0,0,2,1.111487,1.176995,0.811017,None,1.0,Soft
1,1,2017,Brazilian Grand Prix,bottas,1.0,4.0,8.0,0.0,0.0,0,2,1.252126,1.054087,0.942624,None,1.0,Soft
2,2,2017,Brazilian Grand Prix,raikkonen,1.0,3.0,9.0,0.0,0.0,0,2,1.009876,1.124674,0.901477,None,1.0,Soft
3,3,2017,Brazilian Grand Prix,hamilton,1.0,4.0,8.0,0.0,0.0,0,2,0.851359,0.882515,1.063797,None,1.0,Soft
4,4,2017,Brazilian Grand Prix,verstappen,1.0,3.0,9.0,0.0,0.0,0,2,1.020470,0.911364,0.946199,Super soft,1.0,Soft
5,5,2017,Brazilian Grand Prix,ricciardo,1.0,4.0,8.0,0.0,0.0,0,2,1.045744,1.010958,1.172284,Super soft,1.0,Soft
6,6,2017,Brazilian Grand Prix,mass,1.0,2.0,10.0,0.0,0.0,0,2,1.000000,1.000000,1.000000,None,1.0,None
7,7,2017,Brazilian Grand Prix,alonso,1.0,3.0,9.0,0.0,0.0,0,2,0.677418,1.018829,1.069176,None,1.0,None
8,8,2017,Brazilian Grand Prix,perez,1.0,4.0,8.0,0.0,0.0,0,2,1.179297,1.215491,1.082196,None,1.0,Soft
9,9,2017,Brazilian Grand Prix,hulkenberg,1.0,2.0,10.0,0.0,0.0,0,2,1.037634,0.961426,1.072898,None,1.0,None


#### Try classifiers and sampling techniques without tuning hyperparameters to see which method is most promising

In [186]:
njobs = 4
rs=12

In [450]:
# create a dummy class for no sampling
class NoSampling(object):
    
    def fit_sample(self, X, y):
        return X, y

# try out a bunch of classifiers to see which is worth pursuing further
classifiers = [
    ['Logistic Regression', LogisticRegressionCV(multi_class='multinomial', solver='newton-cg', penalty='l2', max_iter= 500, Cs=10, cv=5, verbose=1, n_jobs=njobs, random_state=rs)],
    ['Random Forest', RandomForestClassifier(random_state=rs)],
    ['KNN', KNeighborsClassifier(n_jobs=njobs)],
    ['SVM (linear kernel)', SVC(kernel = 'linear', probability=True, verbose=1, random_state=rs)],
    ['SVM (rbf kernel)', SVC(kernel = 'rbf', probability=True, verbose=1, random_state=rs)]
]

# list out the sampling techniques that we are comparing
samplers = [
    ['No Sampling', NoSampling()],
    ['Random Oversampler', RandomOverSampler(random_state=rs)],
    ['SMOTE', SMOTE(ratio='minority', k_neighbors=3, random_state=rs)],
    ['SMOTE Tomek', SMOTETomek(random_state=rs)],
    ['SMOTE ENN', SMOTEENN(random_state=rs)]
]

In [571]:
def sampling_classifier_loop(train_X, test_X, train_y, test_y):

    # initialise dataframe for capturing coefficients of each feature
    l_classifier = []
    l_resampler = []
    l_recall = []
    l_precision = []

    # loop through all the classifiers except the logreg with class weights
    for classifier in classifiers[1:4]:
        #print '------------------------'
        #print classifier[0].upper()
        #print '------------------------'

        # loop through all the types of samplers
        for sampler in samplers:

            #print 
            #print sampler[0]
            #print '------------------------'

            # perform sampling
            train_X_sm, train_y_sm = sampler[1].fit_sample(train_X, train_y)

            # return dataframe of sampling results
            #print 'Ratio of training set:'
            #print pd.Series(train_y_sm).value_counts() /  pd.Series(train_y_sm).value_counts().sum()

            # fit model on train set
            classifier[1].fit(train_X_sm, train_y_sm)

            # use model to predict y
            y_pred = classifier[1].predict(test_X)

            # display confusion matrix & classification report
            #print 'Confusion Matrix:'
            #print confusion_matrix(test_y, y_pred)
            #print classification_report(test_y, y_pred)

            # save
            l_recall.append(recall_score(test_y, y_pred, average=None))
            l_precision.append(precision_score(test_y, y_pred, average=None))
            l_classifier.append(classifier[0])
            l_resampler.append(sampler[0])

    return l_recall, l_precision, l_classifier, l_resampler

In [390]:
# identify the combination of classifier and resampler that obtains the best recall
def results(l_recall, l_precision, l_classifier, l_resampler, Rcols, Pcols):
    df_results = pd.DataFrame({'Classifier': l_classifier, 'Resampler':l_resampler, 
                              'Recall':l_recall, 'Precision': l_precision})
    df_results = df_results[['Classifier', 'Resampler', 'Recall', 'Precision']]
    df_results[Rcols] = pd.DataFrame(df_results.Recall.values.tolist(), index= df_results.index)
    df_results[Pcols] = pd.DataFrame(df_results.Precision.values.tolist(), index= df_results.index)
    df_results = df_results.drop(['Recall', 'Precision'], axis=1)
    
    return df_results

In [451]:
# Combi 1 stint 1:
s1c1_recall, s1c1_precision, s1c1_classifier, s1c1_resampler = sampling_classifier_loop(X1_c1, X1_c1_17, Y1_c1, Y1_c1_17)

Rcols = ['recall: M', 'recall: None','recall: Soft', 'recall: SS']
Pcols = ['precision: M', 'precision: None','precision: Soft', 'precision: SS']
df_results_s1c1 = results(s1c1_recall, s1c1_precision, s1c1_classifier, s1c1_resampler, Rcols, Pcols)
df_results_s1c1

# KNN + SMOTEK gave the best results

[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.5s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.4s finished


[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

,Classifier,Resampler,recall: M,recall: None,recall: Soft,recall: SS,precision: M,precision: None,precision: Soft,precision: SS
0,Logistic Regression,No Sampling,0.0,0.000000,1.000000,NaN,0.0,0.000000,0.787500,NaN
1,Logistic Regression,Random Oversampler,0.0,0.400000,0.448276,0.476190,0.0,0.083333,0.276596,0.789474
2,Logistic Regression,SMOTE,0.0,0.000000,0.448276,0.642857,0.0,0.000000,0.276596,0.826531
3,Logistic Regression,SMOTE Tomek,0.0,0.200000,0.413793,0.484127,0.0,0.040000,0.266667,0.802632
4,Logistic Regression,SMOTE ENN,0.0,0.400000,0.517241,0.309524,0.0,0.050000,0.283019,0.847826
5,Random Forest,No Sampling,0.0,0.000000,0.448276,0.619048,0.0,0.000000,0.232143,0.795918
6,Random Forest,Random Oversampler,0.0,0.200000,0.448276,0.634921,0.0,0.142857,0.260000,0.808081
7,Random Forest,SMOTE,0.0,0.200000,0.586207,0.507937,0.0,0.200000,0.269841,0.842105
8,Random Forest,SMOTE Tomek,0.0,0.200000,0.517241,0.317460,0.0,0.025641,0.254237,0.784314
9,Random Forest,SMOTE ENN,0.0,0.400000,0.413793,0.182540,0.0,0.031250,0.266667,0.851852


In [452]:
# Combi 1 stint 2:
s2c1_recall, s2c1_precision, s2c1_classifier, s2c1_resampler = sampling_classifier_loop(X2_c1, X2_c1_17, Y2_c1, Y2_c1_17)
df_results_s2c1 = results(s2c1_recall, s2c1_precision, s2c1_classifier, s2c1_resampler, Rcols, Pcols)
df_results_s2c1

# KNN + SMOTE gave the best results

[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.3s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.3s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.3s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.3s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.3s finished


[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

,Classifier,Resampler,recall: M,recall: None,recall: Soft,recall: SS,precision: M,precision: None,precision: Soft,precision: SS
0,Logistic Regression,No Sampling,0.0,0.00,1.000000,0.000000,0.000000,0.000000,0.662500,0.000000
1,Logistic Regression,Random Oversampler,0.0,0.20,0.254717,0.090909,0.000000,0.090909,0.613636,0.103448
2,Logistic Regression,SMOTE,0.0,0.20,0.698113,0.000000,0.000000,0.095238,0.732673,0.000000
3,Logistic Regression,SMOTE Tomek,0.0,0.20,0.311321,0.060606,0.000000,0.129032,0.622642,0.071429
4,Logistic Regression,SMOTE ENN,0.0,0.00,0.386792,0.727273,0.000000,0.000000,0.719298,0.240000
5,Random Forest,No Sampling,0.0,0.05,0.830189,0.060606,0.000000,0.111111,0.692913,0.500000
6,Random Forest,Random Oversampler,0.0,0.15,0.688679,0.030303,0.000000,0.750000,0.708738,0.250000
7,Random Forest,SMOTE,0.0,0.55,0.452830,0.000000,0.000000,0.154930,0.716418,0.000000
8,Random Forest,SMOTE Tomek,1.0,0.50,0.216981,0.060606,0.014286,0.263158,0.718750,0.100000
9,Random Forest,SMOTE ENN,0.0,0.45,0.018868,0.272727,0.000000,0.142857,0.400000,0.264706


In [572]:
# Combi 1 stint 3:
s3c1_recall, s3c1_precision, s3c1_classifier, s3c1_resampler = sampling_classifier_loop(X3_c1, X3_c1_17, Y3_c1, Y3_c1_17)
df_results_s3c1 = results(s3c1_recall, s3c1_precision, s3c1_classifier, s3c1_resampler, Rcols, Pcols)
df_results_s3c1

# KNN + SMOTE gave the best results

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

,Classifier,Resampler,recall: M,recall: None,recall: Soft,recall: SS,precision: M,precision: None,precision: Soft,precision: SS
0,Random Forest,No Sampling,0.0,0.318182,0.181818,0.205128,0.0,0.648148,0.048780,0.347826
1,Random Forest,Random Oversampler,0.0,0.254545,0.272727,0.051282,0.0,0.651163,0.056604,0.125000
2,Random Forest,SMOTE,0.0,0.290909,0.090909,0.179487,0.0,0.581818,0.038462,0.170732
3,Random Forest,SMOTE Tomek,0.0,0.163636,0.090909,0.461538,0.0,0.720000,0.111111,0.268657
4,Random Forest,SMOTE ENN,0.0,0.000000,0.000000,0.717949,0.0,0.000000,0.000000,0.269231
5,KNN,No Sampling,0.0,0.463636,0.545455,0.076923,0.0,0.698630,0.146341,0.428571
6,KNN,Random Oversampler,0.0,0.381818,0.090909,0.282051,0.0,0.700000,0.043478,0.275000
7,KNN,SMOTE,0.0,0.354545,0.363636,0.461538,0.0,0.780000,0.142857,0.346154
8,KNN,SMOTE Tomek,0.0,0.327273,0.090909,0.282051,0.0,0.705882,0.040000,0.275000
9,KNN,SMOTE ENN,0.0,0.000000,0.000000,0.871795,0.0,0.000000,0.000000,0.295652


### Print prediction results

In [566]:
def print_pred_results(dftest, X_train, X_test, Y_train, Y_test, sampler, classifier):
    
    cXs_train_sm, cy_train_sm = sampler.fit_sample(X_train, Y_train)

    classifier.fit(cXs_train_sm, cy_train_sm)
    y_pred = classifier.predict(X_test)

    # Merge test prediction results back to test set
    p = pd.DataFrame(y_pred).reset_index(drop=True)
    results = pd.concat([dftest, p], axis=1)
    
    print "Train Accuracy :: ", accuracy_score(cy_train_sm, classifier.predict(cXs_train_sm))
    print "Test Accuracy :: ", accuracy_score(Y_test, y_pred)

    print 'Confusion Matrix:'
    print confusion_matrix(Y_test, y_pred)
    print classification_report(Y_test, y_pred) 
    
    return results

In [552]:
smote = SMOTE(ratio='minority', k=3, random_state=rs)
smoteen = SMOTEENN( random_state=rs)
smotetomek = SMOTETomek(random_state=rs)
knn = KNeighborsClassifier(n_jobs=njobs)
logreg = LogisticRegressionCV(multi_class='multinomial', solver='newton-cg', penalty='l2', max_iter= 500, Cs=10, cv=5, verbose=1, n_jobs=njobs, random_state=rs)

In [567]:
results_2017_c1_s1 = print_pred_results(XY1_c1_17, X1_c1, X1_c1_17, Y1_c1, Y1_c1_17, smotetomek, knn)
results_2017_c1_s1

Train Accuracy ::  0.858565737052
Test Accuracy ::  0.43125
Confusion Matrix:
[[ 0  0  0  0]
 [ 0  4  0  1]
 [ 0  3 13 13]
 [ 8 37 29 52]]
             precision    recall  f1-score   support

     Medium       0.00      0.00      0.00         0
       None       0.09      0.80      0.16         5
       Soft       0.31      0.45      0.37        29
 Super soft       0.79      0.41      0.54       126

avg / total       0.68      0.43      0.50       160



,index,year,name,driverRef,Medium,Soft,Super Soft,Ultra soft,Hard,clusters(pit strategy),track info clusters,M_m,S_m,SS_m,first set,mandatory combi,0
0,0,2017,Brazilian Grand Prix,vettel,1.0,3.0,9.0,0.0,0.0,0,2,1.111487,1.176995,0.811017,Super soft,1.0,Soft
1,1,2017,Brazilian Grand Prix,bottas,1.0,4.0,8.0,0.0,0.0,0,2,1.252126,1.054087,0.942624,Super soft,1.0,Super soft
2,2,2017,Brazilian Grand Prix,raikkonen,1.0,3.0,9.0,0.0,0.0,0,2,1.009876,1.124674,0.901477,Super soft,1.0,Soft
3,3,2017,Brazilian Grand Prix,hamilton,1.0,4.0,8.0,0.0,0.0,0,2,0.851359,0.882515,1.063797,Soft,1.0,Soft
4,4,2017,Brazilian Grand Prix,verstappen,1.0,3.0,9.0,0.0,0.0,0,2,1.020470,0.911364,0.946199,Super soft,1.0,None
5,5,2017,Brazilian Grand Prix,ricciardo,1.0,4.0,8.0,0.0,0.0,0,2,1.045744,1.010958,1.172284,Soft,1.0,Super soft
6,6,2017,Brazilian Grand Prix,mass,1.0,2.0,10.0,0.0,0.0,0,2,1.000000,1.000000,1.000000,None,1.0,None
7,7,2017,Brazilian Grand Prix,alonso,1.0,3.0,9.0,0.0,0.0,0,2,0.677418,1.018829,1.069176,Super soft,1.0,Soft
8,8,2017,Brazilian Grand Prix,perez,1.0,4.0,8.0,0.0,0.0,0,2,1.179297,1.215491,1.082196,Super soft,1.0,Soft
9,9,2017,Brazilian Grand Prix,hulkenberg,1.0,2.0,10.0,0.0,0.0,0,2,1.037634,0.961426,1.072898,Super soft,1.0,Super soft


In [568]:
results_2017_c1_s2 = print_pred_results(XY2_c1_17, X2_c1, X2_c1_17, Y2_c1, Y2_c1_17, smote, knn)
results_2017_c1_s2

/Users/dianaow/anaconda2/lib/python2.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'k' is deprecated from 0.2 and will be removed in 0.4. Use 'k_neighbors' instead.
  category=DeprecationWarning)


Train Accuracy ::  0.728021978022
Test Accuracy ::  0.44375
Confusion Matrix:
[[ 0  0  1  0]
 [ 3 11  5  1]
 [22 23 60  1]
 [ 5 12 16  0]]
             precision    recall  f1-score   support

     Medium       0.00      0.00      0.00         1
       None       0.24      0.55      0.33        20
       Soft       0.73      0.57      0.64       106
 Super soft       0.00      0.00      0.00        33

avg / total       0.51      0.44      0.46       160



,index,year,name,driverRef,Medium,Soft,Super Soft,Ultra soft,Hard,clusters(pit strategy),track info clusters,M_m,S_m,SS_m,second set,mandatory combi,0
0,0,2017,Brazilian Grand Prix,vettel,1.0,3.0,9.0,0.0,0.0,0,2,1.111487,1.176995,0.811017,Soft,1.0,Medium
1,1,2017,Brazilian Grand Prix,bottas,1.0,4.0,8.0,0.0,0.0,0,2,1.252126,1.054087,0.942624,Soft,1.0,Medium
2,2,2017,Brazilian Grand Prix,raikkonen,1.0,3.0,9.0,0.0,0.0,0,2,1.009876,1.124674,0.901477,Soft,1.0,Medium
3,3,2017,Brazilian Grand Prix,hamilton,1.0,4.0,8.0,0.0,0.0,0,2,0.851359,0.882515,1.063797,Super soft,1.0,Medium
4,4,2017,Brazilian Grand Prix,verstappen,1.0,3.0,9.0,0.0,0.0,0,2,1.020470,0.911364,0.946199,Soft,1.0,None
5,5,2017,Brazilian Grand Prix,ricciardo,1.0,4.0,8.0,0.0,0.0,0,2,1.045744,1.010958,1.172284,Soft,1.0,Medium
6,6,2017,Brazilian Grand Prix,mass,1.0,2.0,10.0,0.0,0.0,0,2,1.000000,1.000000,1.000000,None,1.0,None
7,7,2017,Brazilian Grand Prix,alonso,1.0,3.0,9.0,0.0,0.0,0,2,0.677418,1.018829,1.069176,Soft,1.0,Medium
8,8,2017,Brazilian Grand Prix,perez,1.0,4.0,8.0,0.0,0.0,0,2,1.179297,1.215491,1.082196,Soft,1.0,Medium
9,9,2017,Brazilian Grand Prix,hulkenberg,1.0,2.0,10.0,0.0,0.0,0,2,1.037634,0.961426,1.072898,Soft,1.0,Medium


In [579]:
results_2017_c1_s3 = print_pred_results(XY3_c1_17, X3_c1, X3_c1_17, Y3_c1, Y3_c1_17, smote, knn)
results_2017_c1_s3

/Users/dianaow/anaconda2/lib/python2.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'k' is deprecated from 0.2 and will be removed in 0.4. Use 'k_neighbors' instead.
  category=DeprecationWarning)


Train Accuracy ::  0.635087719298
Test Accuracy ::  0.38125
Confusion Matrix:
[[ 0  0  0  0]
 [25 39 17 29]
 [ 1  1  4  5]
 [ 4 10  7 18]]
             precision    recall  f1-score   support

     Medium       0.00      0.00      0.00         0
       None       0.78      0.35      0.49       110
       Soft       0.14      0.36      0.21        11
 Super soft       0.35      0.46      0.40        39

avg / total       0.63      0.38      0.45       160



,index,year,name,driverRef,Medium,Soft,Super Soft,Ultra soft,Hard,clusters(pit strategy),track info clusters,M_m,S_m,SS_m,third set,mandatory combi,0
0,0,2017,Brazilian Grand Prix,vettel,1.0,3.0,9.0,0.0,0.0,0,2,1.111487,1.176995,0.811017,None,1.0,None
1,1,2017,Brazilian Grand Prix,bottas,1.0,4.0,8.0,0.0,0.0,0,2,1.252126,1.054087,0.942624,None,1.0,Soft
2,2,2017,Brazilian Grand Prix,raikkonen,1.0,3.0,9.0,0.0,0.0,0,2,1.009876,1.124674,0.901477,None,1.0,None
3,3,2017,Brazilian Grand Prix,hamilton,1.0,4.0,8.0,0.0,0.0,0,2,0.851359,0.882515,1.063797,None,1.0,None
4,4,2017,Brazilian Grand Prix,verstappen,1.0,3.0,9.0,0.0,0.0,0,2,1.020470,0.911364,0.946199,Super soft,1.0,Super soft
5,5,2017,Brazilian Grand Prix,ricciardo,1.0,4.0,8.0,0.0,0.0,0,2,1.045744,1.010958,1.172284,Super soft,1.0,Soft
6,6,2017,Brazilian Grand Prix,mass,1.0,2.0,10.0,0.0,0.0,0,2,1.000000,1.000000,1.000000,None,1.0,Super soft
7,7,2017,Brazilian Grand Prix,alonso,1.0,3.0,9.0,0.0,0.0,0,2,0.677418,1.018829,1.069176,None,1.0,None
8,8,2017,Brazilian Grand Prix,perez,1.0,4.0,8.0,0.0,0.0,0,2,1.179297,1.215491,1.082196,None,1.0,None
9,9,2017,Brazilian Grand Prix,hulkenberg,1.0,2.0,10.0,0.0,0.0,0,2,1.037634,0.961426,1.072898,None,1.0,Soft


### Findings: Classfication results indicate overfit as training accuracy is much higher than that of test accuracy. Furthermore , precision and recall scores are not ideal. Perhaps, the feature selection could be fine-tuned further to enable better classification results